<a href="https://colab.research.google.com/github/rll/deepul/blob/master/homeworks/hw1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started

## Overview
This semester, all homeworks will be conducted through Google Colab notebooks. All code for the homework assignment will be written and run in this notebook. Running in Colab will automatically provide a GPU, but you may also run this notebook locally by following [these instructions](https://research.google.com/colaboratory/local-runtimes.html) if you wish to use your own GPU.

You will save images in the notebooks to use and fill out a given LaTeX template which will be submitted to Gradescope, along with your notebook code.

## Using Colab
On the left-hand side, you can click the different icons to see a Table of Contents of the assignment, as well as local files accessible through the notebook.

Make sure to go to **Runtime -> Change runtime type** and select **GPU** as the hardware accelerator. This allows you to use a GPU. Run the cells below to get started on the assignment. Note that a session is open for a maximum of 12 hours, and using too much GPU compute may result in restricted access for a short period of time. Please start the homework early so you have ample time to work.

**If you load this notebook by clicking "Open in Colab" from github, you will need to save it to your own Google Drive to keep your work.**

## General Tips
In each homework problem, you will implement autoregressive models and run it on various datasets. Oftentime you will run it on two datasets (dataset 1 and dataset 2). In these cases, the expected outputs for dataset 1 are already provided to help as a sanity check.

Feel free to print whatever output (e.g. debugging code, training code, etc) you want, as the graded submission will be the submitted pdf with images.

After you complete the assignment, download all of the images outputted in the results/ folder and upload them to the figure folder in the given latex template.

There is a lot of freedom in this homework to design write and design your own models. Hyperparameters are given as a guide to show what worked for us, but feel free to explore and use what you find is best!

Run the cells below to download and load up the starter code.

In [ ]:
%pip install ../..
%pip install matplotlib

In [11]:
import numpy as np
from deepul.hw1_helper import (
    # Q1
    visualize_q1_data,
    q1_sample_data_1,
    q1_sample_data_2,
    q1_save_results,
    # Q2
    q2a_save_results,
    q2b_save_results,
    visualize_q2a_data,
    visualize_q2b_data,
    # Q3
    q3ab_save_results,
    q3c_save_results,
    # Q4
    q4a_save_results,
    q4b_save_results,
    # Q5
    visualize_q5_data,
    q5a_save_results,
    # Q6
    visualize_q6_data,
    q6a_save_results,
)

# Question 1: 1D Data

In this question, we will train simple generative models on discrete 1D data.

Execute the cell below to visualize our datasets

In [ ]:
visualize_q1_data(dset_type=1)
visualize_q1_data(dset_type=2)

## Part (a) Fitting a Histogram

Let $\theta = (\theta_0, \dots, \theta_{d-1}) \in \mathbb{R}^{d}$ and define the model $p_\theta(x) = \frac{e^{\theta_x}}{\sum_{x'}e^{\theta_{x'}}}$

Fit $p_\theta$ with maximum likelihood via stochastic gradient descent on the training set, using $\theta$ initialized to zero. Use your favorite version of stochastic gradient descent, and optimize your hyperparameters on a validation set of your choice.

**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. Plot the model probabilities in a bar graph with $\{0,\dots,d-1\}$ on the x-axis and a real number in $[0,1]$ on the y-axis.




Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("mps")

def q1_a(train_data, test_data, d, dset_id):
  """
  train_data: An (n_train,) numpy array of integers in {0, ..., d-1}
  test_data: An (n_test,) numpy array of integers in {0, .., d-1}
  d: The number of possible discrete values for random variable x
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
             used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (d,) of model probabilities
  """
  
  # 设置超参数
  if dset_id == 1:
    batch_size = 64
    learning_rate = 0.01
    num_epochs = 100
  else:
    batch_size = 128
    learning_rate = 0.01
    num_epochs = 200
    
  # 将数据转换为PyTorch tensors
  train_tensor = torch.LongTensor(train_data)
  test_tensor = torch.LongTensor(test_data)
  
  # 创建数据加载器
  train_dataset = TensorDataset(train_tensor)
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  
  # 初始化参数θ为零
  theta = torch.zeros(d, requires_grad=True)
  
  # 使用Adam优化器
  optimizer = optim.Adam([theta], lr=learning_rate)
  
  # 用于存储损失
  train_losses = []
  test_losses = []
  
  # 计算初始测试损失
  with torch.no_grad():
    test_logits = theta[test_tensor]
    test_log_probs = torch.log_softmax(theta, dim=0)
    test_nll = -test_log_probs[test_tensor].mean()
    test_losses.append(test_nll.item())
  
  # 训练循环
  for epoch in range(num_epochs):
    for batch_idx, (batch_data,) in enumerate(train_loader):
      optimizer.zero_grad()
      
      # 计算对数概率
      log_probs = torch.log_softmax(theta, dim=0)
      
      # 计算负对数似然损失
      nll_loss = -log_probs[batch_data].mean()
      
      # 反向传播
      nll_loss.backward()
      optimizer.step()
      
      # 记录训练损失
      train_losses.append(nll_loss.item())
    
    # 每个epoch后计算测试损失
    with torch.no_grad():
      test_log_probs = torch.log_softmax(theta, dim=0)
      test_nll = -test_log_probs[test_tensor].mean()
      test_losses.append(test_nll.item())
  
  # 计算最终的概率分布
  with torch.no_grad():
    distribution = torch.softmax(theta, dim=0).numpy()
  
  return train_losses, test_losses, distribution

### Results

Once you've implemented `q1_a`, execute the cells below to visualize and save your results



In [ ]:
q1_save_results(1, 'a', q1_a)

In [ ]:
q1_save_results(2, 'a', q1_a)

## Part (b) Fitting Discretized Mixture of Logistics

Let us model $p_\theta(x)$ as a **discretized** mixture of 4 logistics such that $p_\theta(x) = \sum_{i=1}^4 \pi_i[\sigma((x+0.5 - \mu_i)/s_i) - \sigma((x-0.5-\mu_i)/s_i)]$

For the edge case of when $x = 0$, we replace $x-0.5$ by $-\infty$, and for $x = 99$, we replace $x+0.5$ by $\infty$.

You may find the [PixelCNN++](https://arxiv.org/abs/1701.05517) helpful for more information on discretized mixture of logistics.

**Provide the same set of corresponding deliverables as part (a)**

Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [2]:
def q1_b(train_data, test_data, d, dset_id):
  """
  train_data: An (n_train,) numpy array of integers in {0, ..., d-1}
  test_data: An (n_test,) numpy array of integers in {0, .., d-1}
  d: The number of possible discrete values for random variable x
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (d,) of model probabilities
  """
  # 检查MPS可用性
  if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device")
  else:
    device = torch.device("cpu")
    print("MPS not available, using CPU")
  
  device = torch.device("cpu")
  
  # 超参数
  lr = 0.01
  if dset_id == 2:
    lr = 0.05
  num_epochs = 1000
  if dset_id == 2:
    num_epochs = 250
  batch_size = 64
  if dset_id == 2:
    batch_size = 200
  
  # 初始化参数并移到MPS设备
  pi_logits = torch.zeros((4,), dtype=torch.float32, requires_grad=True, device=device)
  s_logits = torch.zeros((4,), dtype=torch.float32, requires_grad=True, device=device)
  mu = torch.zeros((4,), dtype=torch.float32, requires_grad=True, device=device)
  
  # 数据移到设备
  train_data = torch.LongTensor(train_data).to(device)
  test_data = torch.LongTensor(test_data).to(device)
  
  train_set = TensorDataset(train_data)
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
  
  optimizer = optim.Adam([pi_logits, s_logits, mu], lr=lr)
  train_losses, test_losses = [], []
  
  # 计算初始测试损失
  with torch.no_grad():
    pi = torch.softmax(pi_logits, dim=0)
    s = torch.exp(s_logits)
    loss = 0.0
    for x in test_data:
      x_float = x.float()
      if x == 0:
        sigma_min = torch.tensor(0.0, device=device)
        sigma_max = torch.sum(pi * torch.sigmoid((x_float + 0.5 - mu) / s))
      elif x == d-1:
        sigma_max = torch.sum(pi)
        sigma_min = torch.sum(pi * torch.sigmoid((x_float - 0.5 - mu) / s))
      else:
        sigma_max = torch.sum(pi * torch.sigmoid((x_float + 0.5 - mu) / s)) 
        sigma_min = torch.sum(pi * torch.sigmoid((x_float - 0.5 - mu) / s))
      
      prob = sigma_max - sigma_min
      prob = torch.clamp(prob, min=1e-8)
      loss += -torch.log(prob)
    loss /= len(test_data)
    test_losses.append(loss.item())
  
  for epoch in range(num_epochs):
    for batch_idx, (batch_data,) in enumerate(train_loader):
      batch_data = batch_data.to(device)  # 确保batch数据在正确设备上
      optimizer.zero_grad()
      
      pi = torch.softmax(pi_logits, dim=0)
      s = torch.exp(s_logits)
      
      loss = torch.tensor(0.0, device=device)
      for x in batch_data:
        x_float = x.float()
        if x == 0:
          sigma_min = torch.tensor(0.0, device=device)
          sigma_max = torch.sum(pi * torch.sigmoid((x_float + 0.5 - mu) / s))
        elif x == d-1:
          sigma_max = torch.sum(pi)
          sigma_min = torch.sum(pi * torch.sigmoid((x_float - 0.5 - mu) / s))
        else:
          sigma_max = torch.sum(pi * torch.sigmoid((x_float + 0.5 - mu) / s)) 
          sigma_min = torch.sum(pi * torch.sigmoid((x_float - 0.5 - mu) / s))
        
        prob = sigma_max - sigma_min
        prob = torch.clamp(prob, min=1e-8)
        loss += -torch.log(prob)
      loss /= len(batch_data)
      
      train_losses.append(loss.item())
      loss.backward()
      optimizer.step()
    
    # 计算epoch结束后的测试损失
    with torch.no_grad():
      pi = torch.softmax(pi_logits, dim=0)
      s = torch.exp(s_logits)
      loss = torch.tensor(0.0, device=device)
      for x in test_data:
        x_float = x.float()
        if x == 0:
          sigma_min = torch.tensor(0.0, device=device)
          sigma_max = torch.sum(pi * torch.sigmoid((x_float + 0.5 - mu) / s))
        elif x == d-1:
          sigma_max = torch.sum(pi)
          sigma_min = torch.sum(pi * torch.sigmoid((x_float - 0.5 - mu) / s))
        else:
          sigma_max = torch.sum(pi * torch.sigmoid((x_float + 0.5 - mu) / s)) 
          sigma_min = torch.sum(pi * torch.sigmoid((x_float - 0.5 - mu) / s))
        
        prob = sigma_max - sigma_min
        prob = torch.clamp(prob, min=1e-8)
        loss += -torch.log(prob)
      loss /= len(test_data)
      test_losses.append(loss.item())
    
    if epoch % 10 == 0:
      print(f"epoch: {epoch}, train loss: {train_losses[-1]:.4f}, test loss: {test_losses[-1]:.4f}")
  
  # 计算最终概率分布
  distribution = []
  with torch.no_grad():
    pi = torch.softmax(pi_logits, dim=0)
    s = torch.exp(s_logits)
    for x in range(d):
      x_tensor = torch.tensor(float(x), device=device)
      if x == 0:
        sigma_min = torch.tensor(0.0, device=device)
        sigma_max = torch.sum(pi * torch.sigmoid((x_tensor + 0.5 - mu) / s))
      elif x == d-1:
        sigma_max = torch.sum(pi)
        sigma_min = torch.sum(pi * torch.sigmoid((x_tensor - 0.5 - mu) / s))
      else:
        sigma_max = torch.sum(pi * torch.sigmoid((x_tensor + 0.5 - mu) / s)) 
        sigma_min = torch.sum(pi * torch.sigmoid((x_tensor - 0.5 - mu) / s))
      
      prob = sigma_max - sigma_min
      distribution.append(prob.cpu().item())  # 移回CPU再转换为numpy
    
  return np.array(train_losses), np.array(test_losses), np.array(distribution)

### Results

Once you've implemented `q1_b`, execute the cells below to visualize and save your results



In [ ]:
q1_save_results(1, 'b', q1_b)

In [ ]:
q1_save_results(2, 'b', q1_b)

# Question 2 PixelCNNs

Now, you will train more powerful PixelCNN models on the shapes dataset and MNIST. In addition, we will extend to modeling colored datasets.

Run the cell below to visualize the two datasets binary datasets

In [ ]:
visualize_q2a_data(1)
visualize_q2a_data(2)

## Part (a) PixelCNN on Shapes and MNIST
In this part, implement a simple PixelCNN architecture to model binary MNIST and shapes images (same as Q2(b), but with a PixelCNN).

We recommend the following network design:
* A $7 \times 7$ masked type A convolution
* $5$ $7 \times 7$ masked type B convolutions
* $2$ $1 \times 1$ masked type B convolutions
* Appropriate ReLU nonlinearities in-between
* 64 convolutional filters

And the following hyperparameters:
* Batch size 128
* Learning rate $10^{-3}$
* 10 epochs
* Adam Optimizer (this applies to all PixelCNN models trained in future parts)

Your model should output logits, after which you could apply a sigmoid over 1 logit, or a softmax over two logits (either is fine). It may also help to scale your input to $[-1, 1]$ before running it through the network. 

Training on the shapes dataset should be quick, and MNIST should take around 10 minutes

Checkout the Paper for more details: https://arxiv.org/abs/1601.06759

**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. 100 samples from the final trained model



Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [3]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class MaskConv2d(nn.Conv2d):
    def __init__(self, mask_type, in_channel, out_channel, kernel_size, **kw_args):
        if mask_type not in ('A', 'B'):
            raise ValueError(f"mask_type should be 'A' or 'B', not {mask_type}")
        super().__init__(in_channel, out_channel, kernel_size, **kw_args)
        self.mask_type = mask_type
        self.register_buffer('mask', torch.ones_like(self.weight))

        _, _, Hc, Wc = self.weight.shape
        Hc //= 2
        Wc //= 2

        self.mask[:, :, Hc+1:] = 0
        if self.mask_type == 'B':
            self.mask[:, :, Hc, Wc+1:] = 0
        else:
            self.mask[:, :, Hc, Wc:] = 0

    def forward(self, x):
        with torch.no_grad():
            self.weight.data *= self.mask
        return super().forward(x)

In [4]:

class pixelCNN(nn.Module):
    def __init__(self, input_channels=1, hidden_channels=64, output_channels=2):
        super().__init__()
        layers = [
            MaskConv2d('A', input_channels, hidden_channels, kernel_size=7, padding=3),
            nn.ReLU(),
        ]
        for _ in range(5):
            layers.append(MaskConv2d('B', hidden_channels, hidden_channels, kernel_size=5, padding=2))
            layers.append(nn.ReLU())
        layers += [
            MaskConv2d('B', hidden_channels, hidden_channels, kernel_size=1),
            nn.ReLU(),
            MaskConv2d('B', hidden_channels, output_channels, kernel_size=1),
        ]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [5]:

def sample_pixelcnn(model, image_shape=(28, 28), input_channels=1, device='cpu'):
    model.eval()
    H, W = image_shape
    sample = torch.zeros((1, 1, H, W), device=device)
    
    with torch.no_grad():
        for h in range(H):
            for w in range(W):
                logits = model(sample)
                probs = torch.softmax(logits[0, :, h, w], dim=0)
                pixel = torch.multinomial(probs, 1).float()
                # Convert [0,1] -> [-1,1]
                sample[0, 0, h, w] = (pixel - 0.5) * 2  
    return ((sample[0, 0].cpu().numpy() + 1) / 2).round().astype(np.uint8)

In [6]:

def q2_a(train_data, test_data, image_shape, dset_id):
    """
    train_data: (n_train, H, W, 1) binary images in {0, 1}
    test_data: (n_test, H, W, 1) binary images in {0, 1}
    image_shape: (H, W)
    dset_id: 1 or 2 (for hyperparameter tuning)
    Returns:
    - train_losses: list of losses per batch
    - test_losses: list of losses per epoch
    - samples: (100, H, W, 1) generated images in {0, 1}
    """
    batch_size = 128
    lr = 2e-3
    num_epochs = 20

    device = 'cuda' if torch.backends.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    # Model and optimizer
    pixel_cnn = pixelCNN(input_channels=1, hidden_channels=64).to(device)
    optimizer = optim.Adam(pixel_cnn.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    # Normalize data to [-1, 1]
    train_data = (train_data.astype(np.float32) - 0.5) * 2.0
    test_data = (test_data.astype(np.float32) - 0.5) * 2.0

    # To tensor
    train_tensor = torch.FloatTensor(train_data)
    test_tensor = torch.FloatTensor(test_data)

    train_loader = DataLoader(TensorDataset(train_tensor), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(test_tensor), batch_size=batch_size)

    train_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        pixel_cnn.train()
        for batch_idx, (batch,) in enumerate(train_loader):
            x = batch.to(device)  # -> (B, H, W, 1)
            x = x.permute(0, 3, 1, 2)  # -> (B, 1, H, W)

            targets = ((x + 1) // 2).long().squeeze(1)  # -> (B, H, W)

            logits = pixel_cnn(x)  # -> (B, 2, H, W)

            loss = loss_fn(logits, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

            # if batch_idx % 10 == 0:
            #     print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        # Test phase
        pixel_cnn.eval()
        total_test_loss = 0
        with torch.no_grad():
            for (test_batch,) in test_loader:
                x_test = test_batch.to(device).permute(0, 3, 1, 2)
                targets_test = ((x_test + 1) // 2).long().squeeze(1)
                logits_test = pixel_cnn(x_test)
                loss_test = loss_fn(logits_test, targets_test)
                total_test_loss += loss_test.item()
        test_losses.append(total_test_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Loss: {total_test_loss:.4f}')
    # Generate 100 samples using autoregressive sampling
    pixel_cnn.eval()
    samples = []
    with torch.no_grad():
        for _ in range(10):
            sample = sample_pixelcnn(pixel_cnn, image_shape=image_shape, device=device)
            samples.append(sample)

    samples = np.array(samples).reshape(100, *image_shape, 1)

    return train_losses, test_losses, samples

### Results

Once you've implemented `q2_a`, execute the cells below to visualize and save your results



In [ ]:
q2a_save_results(1, q2_a)

In [ ]:
q2a_save_results(2, q2_a)

## Part (b) PixelCNN on Colored Shapes and MNIST: Independent Color Channels

For the next part, we'll work with color images (shapes and MNIST). Run the cell below to visualize the dataset.

In [ ]:
visualize_q2b_data(1)
visualize_q2b_data(2)

Now, implement a PixelCNN to support RGB color channels (or augment your existing implementation). **First, implement a PixelCNN that assumes color channels as independent.** More formally, we model the following parameterized distribution:

$$p_\theta(x) = \prod_{i=1}^{HW}\prod_{c=1}^C p_\theta(x_i^c | x_{<i})$$

Here are some tips that you may find useful for designing and training these models:
* You will need a 4-way softmax for every prediction, as opposed to a 256-way softmax in the PixelCNN paper, since the dataset is quantized to two bits per color channel
* You can set the number of filters for each convolutions to 120. You can use the ReLU nonlinearity throughout.
* Use a stack of 8 residual block architecture from [Figure 5](https://arxiv.org/abs/1601.06759) but with 7 x 7 masked convolutions in the middle instead of 3 x 3 masked convolutions
* Consider using [layer normalization](https://arxiv.org/abs/1607.06450) to improve performance. However, be careful to maintain the autoregressive property.
* With a learning rate of $10^{-3}$ and a batch size of 128, it should take a few minutes to run on the shapes dataset, and about 50-60 minutes on MNIST.

**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. 100 samples from the final trained model



Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [7]:
class pixelResidualCNNBlock(nn.Module):
    def __init__(self, in_channels, hidden_channels, kernel_size=3, padding=1):
        super(pixelResidualCNNBlock, self).__init__()
        self.conv1 = MaskConv2d('B', in_channels, hidden_channels, 1)
        self.conv2 = MaskConv2d('B', hidden_channels, hidden_channels, kernel_size, padding=padding)
        self.conv3 = MaskConv2d('B', hidden_channels, in_channels, 1)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = nn.ReLU()(out)
        out = self.conv2(out)
        out = nn.ReLU()(out)
        out = self.conv3(out)
        out += residual
        return out
class pixelResidualCNN(nn.Module):
    def __init__(self, input_channels=1, hidden_channels=64, class_per_channel=4):
        super().__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.class_per_channel = class_per_channel
        self.input_conv = MaskConv2d('A', input_channels, 2*hidden_channels, kernel_size=7, padding=3)
        self.residual_blocks = []
        for _ in range(5):
            self.residual_blocks.append(pixelResidualCNNBlock(2*hidden_channels, hidden_channels, kernel_size=7, padding=3))
            self.residual_blocks.append(nn.ReLU())
        self.residual_blocks = nn.Sequential(*self.residual_blocks)
        self.output_conv = MaskConv2d('B', 2*hidden_channels, input_channels*class_per_channel, kernel_size=1)
    def forward(self, x):
        x = self.input_conv(x)
        x = nn.ReLU()(x)
        x = self.residual_blocks(x)
        x = self.output_conv(x)
        x = x.reshape(x.shape[0], self.class_per_channel, self.input_channels, x.shape[2], x.shape[3])  # (B, C*class_per_channel, H, W)
        return x


In [8]:
def sample_pixelcnn_simple(model, image_shape=(28, 28), input_channels=1, class_per_channel=4, device='cpu'):
    """
    使用 PixelCNN 模型进行自回归图像采样（逐像素生成）
    
    参数:
        model: 训练好的 pixelResidualCNN 模型
        image_shape: 图像尺寸 (H, W)
        input_channels: 输入图像通道数 (如灰度图=1, RGB=3)
        class_per_channel: 每个通道的离散类别数 (例如 4 级灰度)
        device: 'cpu' 或 'cuda'
    
    返回:
        生成的图像 numpy 数组，形状为 (H, W, C)，值在 [0, 3] 的 uint8
    """
    model.to(device)
    model.eval()
    H, W = image_shape
    sample = torch.randn((1, input_channels, H, W), device=device)  # 初始化为随机噪声
    sample = (sample - 0.5) * 2.0  # 将随机噪声归一化到 [-1, 1]

    with torch.no_grad():
        for h in range(H):
            for w in range(W):
                # 前向传播得到 logits: shape (1, 4, C, H, W)
                logits = model(sample)

                # 对每个通道单独采样
                for c in range(input_channels):
                    logit_pixel = logits[0, :, c, h, w]  # shape (4,)
                    prob_pixel = torch.softmax(logit_pixel, dim=0)  # 转换为概率
                    pixel = torch.multinomial(prob_pixel, 1).item()  # 采样一个类别

                    # 写入图像（归一化格式 [-1, 1]）
                    sample[0, c, h, w] = (float(pixel) - 1.5) / 1.5  # 归一化到 [-1, 1]

    # 后处理：转换回 [0, 3] 的整数
    sample = sample.cpu().numpy()[0].transpose(1, 2, 0)  # (H, W, C)
    sample = ((sample * 1.5 + 1.5)).clip(0, 3).astype(np.uint8)

    return sample

In [9]:
import matplotlib.pyplot as plt
def q2_b(train_data, test_data, image_shape, dset_id):
    """
    train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
    test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
    image_shape: (H, W, C), height, width, and # of channels of the image
    dset_id: An identifying number of which dataset is given (1 or 2).
    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (100, H, W, C) of samples with values in {0, 1, 2, 3}
    """

    # device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    device = 'cpu'
    print(f"Using device: {device}")

    pixel_net = pixelResidualCNN(input_channels=image_shape[2], hidden_channels=128, class_per_channel=4).to(device)
    
    batch_size = 64
    lr = 1e-3
    num_epochs = 5
    
    optimizer = optim.Adam(pixel_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)
    loss_fn = nn.CrossEntropyLoss()

    # Normalize data
    train_data = (train_data.astype(np.float32) - 1.5) / 1.5
    test_data = (test_data.astype(np.float32) - 1.5) / 1.5
    
    test_tensor = torch.FloatTensor(test_data).to(device)
    train_tensor = torch.FloatTensor(train_data).to(device)
    train_loader = DataLoader(TensorDataset(train_tensor), batch_size=batch_size, shuffle=True)

    train_losses = []
    test_losses = []

    pixel_net.eval()
    with torch.no_grad():
        test_set = test_tensor.permute(0, 3, 1, 2).to(device)  # (n_test, C, H, W)
        targets_test = (test_set * 1.5 + 1.5).long()  # (n_test, C, H, W)
        logits_test = pixel_net(test_set)  # (n_test, 4, C, H, W)

        # Reshape for CE loss
        B, C, H, W = targets_test.shape
        logits_reshaped = logits_test.permute(0, 2, 3, 4, 1).reshape(-1, 4)  # (B*C*H*W, 4)
        targets_reshaped = targets_test.reshape(-1).long()  # (B*C*H*W,)
        initial_loss = loss_fn(logits_reshaped, targets_reshaped).item()
        test_losses.append(initial_loss)

    for epoch in range(num_epochs):
        pixel_net.train()
        for batch_idx, (batch,) in enumerate(train_loader):
            x = batch.to(device)
            x = x.permute(0, 3, 1, 2)  # -> (B, C, H, W)
            targets = (x * 1.5 + 1.5).long()  # -> (B, C, H, W)

            logits = pixel_net(x)  # -> (B, 4, C, H, W)

            # Reshape for cross entropy loss
            B, _, C_, H_, W_ = logits.shape
            logits = logits.permute(0, 2, 3, 4, 1).reshape(-1, 4)
            targets = targets.permute(0, 1, 2, 3).reshape(-1).long()

            loss = loss_fn(logits, targets)
            train_losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        # Test after each epoch
        pixel_net.eval()
        with torch.no_grad():
            test_set = test_tensor.permute(0, 3, 1, 2).to(device)
            targets_test = (test_set * 1.5 + 1.5).long()
            logits_test = pixel_net(test_set)

            B, C, H, W = targets_test.shape
            logits_reshaped = logits_test.permute(0, 2, 3, 4, 1).reshape(-1, 4)
            targets_reshaped = targets_test.reshape(-1).long()
            loss_test = loss_fn(logits_reshaped, targets_reshaped).item()
            test_losses.append(loss_test)

            print(f'Epoch [{epoch+1}/{num_epochs}], Test Loss: {loss_test:.4f}')
        
        scheduler.step()

    # Save model
    torch.save(pixel_net.state_dict(), f'pixel_residual_cnn_{dset_id}.pth')

    # Generate samples
    samples = []
    for _ in range(100):
        sample = sample_pixelcnn_simple(pixel_net, image_shape=image_shape[:2], input_channels=image_shape[2], class_per_channel=image_shape[2])
        samples.append(sample)
    samples = np.array(samples).astype(np.uint8)
    return train_losses, test_losses, samples
    

### Results

Once you've implemented `q2_b`, execute the cells below to visualize and save your results



In [ ]:
q2b_save_results(1, 'b', q2_b)

In [12]:
q2b_save_results(2, 'b', q2_b)

FileNotFoundError: [Errno 2] No such file or directory: 'deepul\\homeworks\\hw1\\data\\mnist_colored.pkl'

# Question 3: Causal Transformer - iGPT

Now we will move onto the current most popular and widespread autoregressive model, the transformer.

## Part (a) Autoregressive Transformer on Shapes and MNIST
In this part, implement a simple Autoregressive Transformer to model binary MNIST and shapes images (same as Q2(a), but with a Transformer). 

Some additional notes about your transformer implementation:
 * iGPT uses learned positional encodings. We recommend to use those here as well. However, you may also use sinusoidal positional encodings if you wish (see the [Attention is All You Need](https://arxiv.org/abs/1706.03762) paper)
 * Autoregressive transformer always predicts the **next** token, give prior tokens. iGPT has a special **\<bos\>** or beginning of sequence token at the start of every sequence every image. Make sure to include this in your implementation as well. You can generate unconditional sample by conditioning with the **\<bos\>** token.
 * While dropout is a common feature in transformer models, you do not need to add it (but may if you wish!).
 * Prebuilt transformers exist in some frameworks (i.e. pytorch). Don't just use an off the shelf implementation as the point of the exercise is to better understand the transformer architecture. Building the transformer from the ground up (use primitives such as Linear/Dense layers, LayerNorm, GeLU, Embedding)
 * Learning rate warmup and cos learning rate decay are often used when training transformers to improve training stability and improve performance. See if this helps your model! Try 1000 steps of warmup with a cosine learning rate decay.

Paper references
* [Attention Is All You Need](https://arxiv.org/abs/1706.03762) 
* [Generative Pretraining from Pixels](https://cdn.openai.com/papers/Generative_Pretraining_from_Pixels_V2.pdf) 
* [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)

We recommend the following network design parameters:
* $d_{model}$: 128
* heads: 4
* layers: 2
* GeLU nonlinearities

And the following hyperparameters:
* Batch size: 64 or 32 or 16 (whichever fits in your GPU)
* Learning rate: $10^{-3}$
* 15 epochs or more
* Adam Optimizer (this applies to all Transformers models trained in future parts)

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. 100 samples from the final trained model



In [ ]:
def q3_a(train_data, test_data, image_shape, dset_id):
  """
  train_data: A (n_train, H, W, 1) uint8 numpy array of color images with values in {0, 1}
  test_data: A (n_test, H, W, 1) uint8 numpy array of color images with values in {0, 1}
  image_shape: (H, W, 1), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, W, 1) of samples with values in {0, 1}
  """
  return train_losses, test_losses, samples

### Results

Once you've implemented `q3_a`, execute the cells below to visualize and save your results

In [ ]:
q3ab_save_results(1, 'a', q3_a)

In [ ]:
q3ab_save_results(2, 'a', q3_a)

## Part (b) iGPT on Colored Shapes and MNIST

Now, implement an iGPT that models color. In order to reduce the length of token sequences, iGPT models each RGB pixel as a **single** token. This effectively reduces the context length from H*W*C to just H*W. iGPT does this through a k-means clustering approach. Because our images only each can only take on 4 values (2 bits) per channel, we can represent each pixel with 64 values (6 bits). Convert the dataset into an image of tokens and train iGPT on the colored shapes and MNIST dataset.

Checkout the iGPT paper for more details: [Generative Pretraining from Pixels](https://cdn.openai.com/papers/Generative_Pretraining_from_Pixels_V2.pdf) 

Training times and hyperparameter settings should be the same as part (a), except train for longer (15 epochs)

**You will provide these deliverables**

1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. 100 samples from the final trained model


In [ ]:
def q3_b(train_data, test_data, image_shape, dset_id):
  """
  train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  image_shape: (H, W, C), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, W, C) of samples with values in {0, 1, 2, 3}
  """
  return train_losses, test_losses, samples

### Results

Once you've implemented `q3_b`, execute the cells below to visualize and save your results

In [ ]:
q3ab_save_results(1, 'b', q3_b)

In [ ]:
q3ab_save_results(2, 'b', q3_b)

## Part (c) K, V Caching for Improved Inference
You may have noticed that generation from the transformer is quite slow. Part of this is just due to the autoregressive nature. However, another part is due to some computational inefficiency. At each forward pass of the model, we are performing repeat computation of the past sequence. Specifically, we can cache the key and values at the multi attention layer to more quickly predict at each step.

In self-attention, a sequence is processed by generating three vectors for each element in the sequence: a Query (Q), a Key (K), and a Value (V). These vectors are then used to compute attention scores and subsequently the output of the attention layer.
Mathematically, this can be represented as:
 * For each index $i$, compute $Q_i$, $K_i$, $V_i$ for the current element
 * Retrieve $K_{<i}$ and $V_{<i}$ from the cache (where $<i$ denotes all indices before the current one)
 * Compute the attention output using $Q_i$, $[K_{<i}, K_i]$, $[V_{<i}, V_i]$


Next implement caching for your transformer to make inference more efficient by modifying your self attention. Use caching for inference in the future problems for faster generation! (Note caching is only used during inference). You will use the same dataset as in part B, dataset 2 of this question (colored mnist). No training is required in this section, feel free to reuse the model you trained in part B, dataset 2.

**You will provide these deliverables**

1. Over the course of inference, measure the time for the forward pass over the total sequence length with and without caching.
3. 100 samples from the final trained model using the caching inference pipeline.



In [ ]:
def q3_c(train_data, test_data, image_shape, dset_id):
  """
  train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  image_shape: (H, W, C), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets

  Returns
  - a (# sampling steps,) numpy array of time per sampling iteration, without caching
  - a (# sampling steps,) numpy array of time per sampling iteration, with without caching
  - a numpy array of size (100, H, C, W) of samples with values in {0, 1, 2, 3} (sample generated without caching)
  - a numpy array of size (100, H, C, W) of samples with values in {0, 1, 2, 3} (sample generated with caching)
  """
  return time_list_no_cache, time_list_with_cache, samples_no_cache, samples_with_cache

### Results

Once you've implemented `q3_c`, execute the cells below to visualize and save your results



In [ ]:
q3c_save_results(2, q3_c)

# Question 4: Causal Transformer: Tokenized Images

## Image Tokenization with Vector Quanization

## Part (a) Image Quantization

Above, we implemented iGPT, which autoregressivly predicts raw pixels. Transformers have quadratic complexity in the sequence length which prevents this naive approach from scaling well to large images.

The space of natural images often contains very correlated information. This suggests we can learn a reduced representation. VQVAE is a method that does just that, learning to map images to a more compact discrete set of tokens. We will cover this method in more detail in future lectures. The only thing you need to know now is that we can learn an encoder (and corresponding decoder), which can extract a discrete representation from an image. 

If you are curious, checkout the VQVAE paper to learn more: https://arxiv.org/abs/1711.00937 (we will cover this in a future lecture though!)

In this part, we provide a pre-trained VQVAE model, which consists of:
 * encoder to tokenize the images
 * the decoder to recover the image
 * a token vocabulary of VQVAE_MODEL.n_embeddings

Below is the code for loading the VQ model. Note that VQVAE encoding process is lossy, so the decoded images will not be the exact same as the input. Some blurriness in the recovered image is to be expected. The docstrings of the relevant methods you will need for the VQVAE_MODEL are provided below for your convenience. 

We will use 2 colored mnist datasets in this part. The first is the same dataset used in previous parts. The second, hads a colored digit on a differently colored background. We will call these datasets Colored MNIST and Colored MNIST v2. Note that the vqvae is trained per dataset.

**You will provide these deliverables**

1. Use the provided encoder model to quantize the images then inspect the recovered images by applying the decoder for each of the two datasets

In [ ]:
# @property
# def n_embeddings(self) -> int:
#     """The size of the token vocabulary"""
#    
# def quantize(self, x: np.ndarray) -> np.ndarray:
#     """Quantize an image x.
#
#     Args:
#         x (np.ndarray, dtype=int): Image to quantize. shape=(batch_size, 28, 28, 3). Values in [0, 3].
#
#     Returns:
#         np.ndarray: Quantized image. shape=(batch_size, 7, 7). Values in [0, n_embeddings]
#     """
#    
# def decode(self, z_index: np.ndarray) -> np.ndarray:
#     """Decode a quantized image.
#
#     Args:
#         z_index (np.ndarray, dtype=int): Quantized image. shape=(batch_size, 7, 7). Values in [0, n_embeddings].
#
#     Returns:
#         np.ndarray: Decoded image. shape=(batch_size, 28, 28, 3). Values in [0, 3].
#     """
# 

In [ ]:
def q4_a(images, vqvae):
  """
  images: (B, H, W, C), the images to pass through the encoder and decoder of the vqvae
  vqvae: a vqvae model, trained on the relevant dataset

  Returns
  - a numpy array of size (2, H, W, C) of the decoded image
  """
  return autoencoded_images

In [ ]:
q4a_save_results(1, q4_a)

In [ ]:
q4a_save_results(2, q4_a)

## Part (b) Autoregressive Transformer on Colored Shapes and MNIST with Vector Quantization

We can use the VQVAE to tokenize an image dataset. This will result in a much smaller sequence length than the approach we tried in Question 3(b). For this part, train a transformer on the dataset tokenized by the VQVAE.

This is a simplified version of the approach used in VQGAN [VQGAN](https://arxiv.org/abs/2012.09841) -> Section 3.2: Learning the Composition of Images with Transformers (Again, we will cover this in more detail in a future lecture!)

Update the following hyperparameters:
* layers: 4 (we can train a bigger transformer now since less memory is used per input!)
* 30 epochs

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. 100 samples from the final trained model

In [ ]:
def q4_b(train_data, test_data, image_shape, dset_id, vqvae):
  """
  train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  image_shape: (H, W, C), height, width, and # of channels of the image
  dset_id: An identifying number of which dataset is given (1 or 2). Most likely
           used to set different hyperparameters for different datasets
  vqvae: a vqvae model, trained on dataset dset_id

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a numpy array of size (100, H, C, W) of samples with values in {0, 1, 2, 3}
  """
  return train_losses, test_losses, samples

### Results

Once you've implemented `q4_b`, execute the cells below to visualize and save your results



In [ ]:
q4b_save_results(1, q4_b)

In [ ]:
q4b_save_results(2, q4_b)

# Question 5: Causal Transformer: Text

Now lets consider text! You are probably already fimilar with autoregressive transformers for text, now more commonly known as Large Language Modesl (LLMs).
We will now implement a simplified version.

We will be detailing with a [small poetry dataset](https://huggingface.co/datasets/merve/poetry). See some of the data below.

In [ ]:
data = visualize_q5_data()

## Part (a) Modeling Text
Train a transformer on the poetry dataset.

Data Preprocessing:
* We will use a simple method to tokenize the data. We will convert each unique character into a token. (Current LLMs use more sophisticated tokenizers, most commonly, [byte-pair encoding](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt))
* Previously we have leveraged a **\<bos\>** as part of the model, just like iGPT. For text, we may not always sample a sequence that starts at the beginning. Instead, we will add the **\<bos\>** token to the beginning of every sequence in the dataset, and remove the **\<bos\>** token from the model.
* Another problem is that the model must know when to stop sampling. This is done by appending an **\<eos\>**, or end of sequence token at the end of every sequence in the dataset.
* We can now convert the sequence into subsequences of size context_length, for training!

We recommend the following hyperparameters:
* Sequence length: 128
* 5 epochs

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. Provide **5 unconditional samples** of **128 characters** showcasing the model text generation capabilities (text samples should stop after **\<eos\>**. Text after **\<eos\>** can be removed in post processing)

In [ ]:
import torch.utils.data as data
def q5_a(train_text, test_text):
  """
  train_text: list[str] Train text sequences.
  test_text: list[str] Test text sequences.

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a list of 5 (str), 5 generated samples from the model.
  """
  return train_losses, test_losses, text_samples

### Results

Once you've implemented `q5_a`, execute the cells below to visualize and save your results



In [ ]:
q5a_save_results(q5_a)

# Question 6: Causal Transformer: Multimodal

So far, we have been dealing only with autoregressive generation of a single modality. Now we will train a model that operates on multiple modalities!

We will use the text labeled colored MNIST dataset, which has a text description of the MNIST image. Run the cell below to visualize the data along with the text annotation. This is the Colored MNIST v2 dataset, which also comes with these text labels.

In [ ]:
visualize_q6_data()

## Part (a) Multimodal Text and Image Generation
Implement and train an autoregressive (AR) model capable of handling both text and image data. The model should be designed to process sequences composed of concatenated text and image tokens in both orders (text followed by images and images followed by text). Additionally, the model should be capable of generating unconditional text and image samples.

Data Preprocessing:
* Text Tokens: Map each unique word in the text data to a unique token. (Note that all text descriptions contain the exact same amount of words. This simplifies text processing, as you won't have to deal with sequences of different lengths as in Question 5)
* Image Tokens: Quantize the image data into tokens using the VQVAE tokenizer from Problem 4.
* In this problem, we have 2 modalities. Introduce an **\<end of text\>** token and an **\<end of image\>** token. After seeing such a token, the model should switch to sampling the next modality.
* Formulate batches as sequences of concat([**\<end of image\>**, text_tokens, **\<end of text\>**, image_tokens]) and concat([**\<end of text\>**, image_tokens, **\<end of image\>**, text_tokens]). With a 50/50 split between each ordering.

Inference:
* During inference, we cannot mix modality tokens. During sampling we can restrict the logits to only be within the relevant modality.
* After **\<end of image\>**, only allow the model to sample text tokens (including **\<end of text\>**)
* After **\<end of text\>**, only allow the model to sample image tokens (including **\<end of image\>**)
* At the very start (conditioned on the **\<bos\>** token, only allow the model to sample one of (**\<end of image\>** or **\<end of text\>**))
* As the model may not always correctly sample the **\<end of image\>** token before the image ends, you may add a rule to force the model to always sample the correct number of image tokens (49 tokens).

You can use the same hyperparameters as in 4(b) (but of course, feel free to tune your model to achieve better performance)

**You will provide these deliverables**

1. Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2. Report the final test set performance of your final model
3. 9 conditional samples based on provided text.
4. 9 conditional samples based on provided images.
5. 9 unconditional samples showcasing the model's capability in generating standalone text and images.

In [ ]:
def q6_a(train_data, test_data, image_shape, train_text, test_text, image_test_prompt, text_test_prompt, vqvae):
  """
  train_data: A (n_train, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  test_data: A (n_test, H, W, C) uint8 numpy array of color images with values in {0, 1, 2, 3}
  image_shape: tuple (H, W, C) The shape of the images in the dataset, indicating height, width, and number of color channels.
  train_text: list[str] Text data associated with each training image.
  test_text: list[str] Text data associated with each test image.
  image_test_prompt: (9, H, W, C) Image data used for generating conditional text samples during testing.
  text_test_prompt: list of 9 strings Text prompts used for generating conditional image samples during testing.
  vqvae: a vqvae model, trained on the relevant dataset

  Returns
  - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
  - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
  - a list of 9 (image, text), corresponding to the image conditioned samples
  - a list of 9 (image, text), corresponding to the text conditions samples
  - a list of 9 (image, text), corresponding to unconditional samples
  """
  return train_losses, test_losses, samples_image_conditioned, samples_text_conditioned, samples_unconditioned

### Results

Once you've implemented `q6_a`, execute the cells below to visualize and save your results



In [ ]:
q6a_save_results(q6_a)